# Lesson 4 - Data Generation
- 이번 실습자료에서는 파이토치 모델에 이미지를 입력값으로 주기위해 전처리를 하는 방법을 배웁니다.
- 파이토치는 torch.utils.data에 있는 Dataset, DataLoader 클래스가 이 작업을 간편하게 해줍니다.
## 0. Libraries & Configurations
1- 시각화에 필요한 라이브러리와 데이터 경로를 설정합니다.

In [1]:
import os
import sys
from sklearn.metrics import f1_score
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
import tqdm
from time import time
import math

import torch
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import resnet18, resnet50, resnet101
from torchvision.models import resnet34
from torchvision.transforms import Normalize, Resize, ToTensor
from torch.optim import lr_scheduler

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from efficientnet_pytorch import EfficientNet
#define

In [2]:
### Configurations
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train_with_label.csv'

In [3]:
df = pd.read_csv(df_path)
df_idx = {"id":0, "gender":1, "age":2, "path":3, "name":4, "label":5}

In [27]:
df.groupby('label').size()

label
0     2745
1     2050
2      415
3     3660
4     4085
5      545
6      549
7      410
8       83
9      732
10     817
11     109
12     549
13     410
14      83
15     732
16     817
17     109
dtype: int64

## 1. Image Statistics

In [4]:
def get_ext(img_dir, img_id):
    filename = os.listdir(os.path.join(img_dir, img_id))[0]
    ext = os.path.splitext(filename)[-1].lower()
    return ext

In [5]:
def get_img_stats(img_ids):
    img_info = dict(heights=[], widths=[], means=[], stds=[])
    for path in tqdm.tqdm(img_ids):
        img = np.array(Image.open(path))
        h, w, _ = img.shape
        img_info['heights'].append(h)
        img_info['widths'].append(w)
        img_info['means'].append(img.mean(axis=(0,1)))
        img_info['stds'].append(img.std(axis=(0,1)))
    return img_info

In [6]:
img_info = get_img_stats(df.path.values)
#print(f'RGB Mean: {np.mean(img_info["means"], axis=0) / 255.}')
#print(f'RGB Standard Deviation: {np.mean(img_info["stds"], axis=0) / 255.}')

100%|██████████| 18900/18900 [06:46<00:00, 46.54it/s]


In [7]:
print(f'RGB Mean: {np.mean(img_info["means"], axis=0) / 255.}')
print(f'RGB Standard Deviation: {np.mean(img_info["stds"], axis=0) / 255.}')

RGB Mean: [0.56019358 0.52410121 0.501457  ]
RGB Standard Deviation: [0.23318603 0.24300033 0.24567522]


## 2.1 Augmentation Function

In [8]:
mean, std = (0.5, 0.5, 0.5), (0.2, 0.2, 0.2)

In [9]:
from albumentations import *
from albumentations.pytorch import ToTensorV2


def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            Resize(img_size[0], img_size[1], p=1.0),
            HorizontalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            GaussNoise(p=0.5),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = Compose([
            Resize(img_size[0], img_size[1]),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

In [10]:
class MaskBaseDataset(data.Dataset):
    class_labels = []
    image_images = []
    
    def __init__(self, data_df, transform=None):
        """
        MaskBaseDataset을 initialize 합니다.

        Args:
            img_dir: 학습 이미지 폴더의 root directory 입니다.
            transform: Augmentation을 하는 함수입니다.
        """
        self.mean = mean
        self.std = std
        self.transform = transform
        self.df = data_df

        self.setup()
        
    def setup(self):
        """
        image의 경로와 각 이미지들의 label을 계산하여 저장해두는 함수입니다.
        """
        for index in tqdm.tqdm(range(self.__len__())):
            df_series = self.df.iloc[index]
            img_path = df_series[df_idx["path"]]
            if os.path.exists(img_path):
                self.image_images.append(self.transform(image=np.array(Image.open(img_path)))['image'])
                self.class_labels.append(df_series[df_idx["label"]])

    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        #image_path = self.image_paths[index]
        image = self.image_images[index]
        class_label = self.class_labels[index]
        
        # 이미지를 Augmentation 시킵니다.
        #image_transform = self.transform(image=np.array(image))['image']
        return image, class_label

    def __len__(self):
        return len(self.df)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=1004)

In [13]:
drop_list = []
for drop_idx in train_df.index:
    if train_df.loc[drop_idx]['label'] == 0:
        if np.random.randint(6) != 0:
            drop_list.append(drop_idx)
    elif train_df.loc[drop_idx]['label'] == 1:
        if np.random.randint(5) != 0:
            drop_list.append(drop_idx)
    elif train_df.loc[drop_idx]['label'] == 3:
        if np.random.randint(9) != 0:
            drop_list.append(drop_idx)
    elif train_df.loc[drop_idx]['label'] == 4:
        if np.random.randint(10) != 0:
            drop_list.append(drop_idx)
    elif train_df.loc[drop_idx]['label'] in [9, 10, 12, 15, 16]:
        if np.random.randint(2) != 0:
            drop_list.append(drop_idx)

In [14]:
drop_train_df = train_df.drop(drop_list)

In [15]:
drop_train_df.groupby('label').size()

label
0     373
1     312
2     324
3     315
4     361
5     437
6     443
7     316
8      64
9     304
10    314
11     87
12    225
13    319
14     67
15    289
16    319
17     91
dtype: int64

In [16]:
# 정의한 Augmentation 함수와 Dataset 클래스 객체를 생성합니다.
start = time()
transform = get_transforms(mean=mean, std=std)

train_dataset = MaskBaseDataset(
    data_df=drop_train_df, transform=transform["train"]
)

val_dataset = MaskBaseDataset(
    data_df=val_df, transform=transform["val"]
)

print(time() - start)

100%|██████████| 3780/3780 [00:29<00:00, 130.33it/s]

167.4860429763794


In [149]:
len(val_dataset)

3780

## 3. DataLoader

In [40]:
# training dataloader은 데이터를 섞어주어야 합니다. (shuffle=True)
train_loader = data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=False
)

In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [42]:
LEARNING_RATE = 0.0001
NUM_EPOCH = 10

In [43]:
def resnet_finetune(classes):
    #model = EfficientNet.from_pretrained('efficientnet-b2', num_classes=classes)
    model = resnet18(pretrained=True)
    # class 18개로 분리
    model.fc = torch.nn.Linear(in_features=512, out_features=classes, bias=True)

    torch.nn.init.xavier_uniform_(model.fc.weight)
    stdv = 1/math.sqrt(512)
    model.fc.bias.data.uniform_(-stdv, stdv)

    return model

In [44]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [45]:
my_resnet = resnet_finetune(18).to(device)
loss_fn = (torch.nn.CrossEntropyLoss())
optimizer = torch.optim.Adam(my_resnet.parameters(), lr=LEARNING_RATE)

In [46]:
dataloaders = {"train": train_loader, "test": val_loader}

In [47]:
#torch.cuda.empty_cache()
f1_best = 0
count = 0
epoch_f1 = 0

In [48]:
for epoch in range(NUM_EPOCH):
    for phase in ["train", "test"]:
        running_loss = 0.0
        running_acc = 0.0
        if phase == "train":
            my_resnet.train()
        elif phase == "test":
            my_resnet.eval()
        
        # 배치 단위로 data load하여서 작업 -> 이때 transpose 및 여러 함수가 적용된다.
        epoch_f1 = 0
        n_iters=0
        for ind, (images, labels) in enumerate(tqdm.tqdm(dataloaders[phase], leave=False)):
            images = torch.stack(list(images), dim=0).to(device)
            labels = torch.tensor(list(labels)).to(device)

            optimizer.zero_grad()  # parameter gradient를 업데이트 전 초기화함

            with torch.set_grad_enabled(
                phase == "train"
            ):  
                logits = my_resnet(images)
                _, preds = torch.max(
                    logits, 1
                )
                loss = loss_fn(logits, labels)

                if phase == "train":
                    loss.backward()  # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient
                    optimizer.step()  # 계산된 gradient를 가지고 모델 업데이트

            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(
                preds == labels.data
            )
            epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            n_iters += 1

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)

        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "test":
            epoch_f1 /= n_iters
            if epoch_f1 > f1_best:
                f1_best = epoch_f1
                count = 0
            else:
                count += 1
            print("best : ", f1_best, ", last : ", epoch_f1)
    if count == 1:
        print("early stop", "-"*20)
        break
print("학습 종료!")

  0%|          | 0/30 [00:00<?, ?it/s]         

현재 epoch-0의 train-데이터 셋에서 평균 Loss : 1.469, 평균 Accuracy : 0.585


  0%|          | 0/39 [00:00<?, ?it/s]         

현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.539, 평균 Accuracy : 0.830
best :  0.709441317260629 , last :  0.709441317260629


  0%|          | 0/30 [00:00<?, ?it/s]         

...

  0%|          | 0/39 [00:00<?, ?it/s]         

현재 epoch-4의 test-데이터 셋에서 평균 Loss : 0.028, 평균 Accuracy : 1.000
best :  1.0 , last :  1.0


  0%|          | 0/30 [00:00<?, ?it/s]         

현재 epoch-5의 train-데이터 셋에서 평균 Loss : 0.024, 평균 Accuracy : 1.000


현재 epoch-5의 test-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 1.000
best :  1.0 , last :  1.0
early stop --------------------
학습 종료!


In [49]:
print("best : ", f1_best, ", last : ", epoch_f1)

best :  1.0 , last :  1.0


In [50]:
eval_dir = '/opt/ml/input/data/eval'
test_dir = "/opt/ml/input/data/eval"
submission = pd.read_csv(os.path.join(test_dir, "info.csv"))
image_dir = os.path.join(test_dir, "images")

In [51]:
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

In [91]:
class TestDataset(data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [92]:
from torchvision import transforms

In [104]:
test_transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
])

In [105]:
dataset = TestDataset(image_paths, test_transform)
loader = data.DataLoader(dataset, shuffle=False)

my_resnet.eval()
all_predictions = []

In [106]:
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = my_resnet(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

print("test inference is done!")
submission

KeyError: 'You have to pass data to augmentations as named arguments, for example: aug(image=image)'

In [163]:
submission.to_csv('/opt/ml/input/data/eval/submission_resnet18_10_0001_256_undersample.csv', index=False)